## This is the only cell you need to change ##

### Required packages ###
- numpy
- pandas

### Possible filter file ###
- Default: Set_filter_file.xlsx
- Any other filter file name as long as file exists in main directory
  
### Possible file formats ###
- CSV
- excel

### Possible output formats ###
- long format where each line contains one value
- wide table-style format where values for different years (and in case of trade parameters regions) are grouped per line

### Possible processing options ###
- both creates the dataset for parameters and timeseries
- parameters_only for only regular parameters (saves time since timeseries, especially with excel, takes a lot of time)
- timeseries_only for only timeseries

### Possible scenario options ###
- Any scenario name. The script will search for input data for the specified this scenario and overwrite the default values.

In [1]:
# Options regarding output format and processing
settings_file = 'Set_filter_file.xlsx' # Specify which filter file should be used
output_file_format = 'excel'  # Change this to 'csv' or 'excel' depending on your needs
output_format = 'wide' # Change to 'wide' or 'long' depending on your needs
processing_option = 'both' # Change to 'parameters_only' if you don't want the timeseries to created (saves time) or 'timeseries_only' for only timeseries or 'both' 
scenario_option = 'None' # Specify your scenario here. Type 'None' for no scenario

## Run this cell to execute the script ##
### You don't need to change anything in here ### 

In [2]:
from functions.function_import import master_function

# Call the function
master_function(settings_file,output_file_format, output_format, processing_option, scenario_option)


FileNotFoundError: No Excel file found for scenario option: 'Scenario1'. Expected: , or 'None'.